<center>
    <div style="font-size: 24px; font-weight: 500; margin-bottom: -10px">
        Climat et Changement Climatique
    </div>
    <h3>Master</h3>
    <h5 style="margin-top: -5px">Sciences de l’Océan, de l’Atmosphère et du Climat</h5>
    <h1>TP - Carbon Cycle and forcings on 20e-century</h1>
    <h2>Part III: Investigating the contribution of various greenhouse gases</h2>
</center>


## 1. Introduction

In our previous work, we have investigated the role of the rising CO$_2$ to the change in surface temperature. Now, let’s have a look to the role of the methane, another powerful greenhouse gas.

In the last IPCC report, [Myrhe et al. (2013a)](https://acp.copernicus.org/articles/13/1853/2013/) employed the impulse response function (IRF) for surface temperature in response to a pulse of methane emission which has been established by [Boucher et Reddy (2008)](https://www.sciencedirect.com/science/article/abs/pii/S0301421507003709?via%3Dihub). It is written as :

$$\text{IRF}_\text{tas}(t) = \frac{0.631}{8.4} \exp\left(\frac{-t}{8.4}\right) + \frac{0.429}{409.5} \exp\left(\frac{-t}{409.5}\right) $$

In the following, we will use this formulation to study the __Global Warming Potential__ (GWP) and the __Global Temperature Potential__ (GTP) of the methane.


## 2. Global Warming Potential (GWP) and Global Temperature Potential (GTP)

The _Absolute Global Warming Potential_ (AGWP) of a chemical species X is defined by the cumulative sum of the radiatif forcing over a temporal horizon (T$_H$) induced by a pulse of emission, with an amplitude $\delta E_\text{pulse}^X$. That is :

$$\text{AGWP}^X(T_H) = \frac{1}{\delta E_\text{pulse}^X} \int_0^{T_H} F_\text{pulse}^X(t)\,dt$$

We use AGWP of a reference species (e.g. $X$=CO$_2$) to determine the Global Warming Potential(GWP) of a chemical species X :

$$\text{GWP}^X(T_H) = \frac{\text{AGWP}^X(T_H)}{\text{AGWP}^{\text{CO}_2}(T_H)}\,\frac{\delta E_\text{pulse}^{\text{C0}_2}}{\delta E_\text{pulse}^X}$$

Similarily, the _Absolute Global Temperature Potential_ (AGTP) and the __Global Temperature Potential__ (GTP) of a chemical species X is defined by the change in global mean temperature over a temporal horizon ($T_H$) resulting from a emission pulse. Both metrics as written as follows :

$$\text{AGTP}^X(T_H) = \frac{1}{\delta E_\text{pulse}^X} \left[tas^X(T_H) - tas(t=0)\right]$$

$$\text{GTP}^X(T_H) = \frac{\text{AGTP}^X(T_H)}{\text{AGTP}^{\text{CO}_2}(T_H)}\,\frac{\delta E_\text{pulse}^{\text{C0}_2}}{\delta E_\text{pulse}^X}$$


### 2.1 Assessing the GWP and the GTP for the methane ($CH_4$)

Along this notebook, we will use CMIP5 model outputs (similar to Part II).  
You first need to import and load required modules:  

In [ ]:
# header: importing useful modules and functions
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fmin

Then, reload the model outputs:

In [ ]:
# ancillary lists for loading data
model_list = ["ACCESS-10","ACCESS-13","BCC-CSM-11",\
    "BCC-CSM-11m","CanESM2","CCSM4","CNRM-CM5",\
    "CNRM-CM5-2","CSIRO-Mk360","GFDL-CM3","GFDL-ESM2G",\
    "GFDL-ESM2M","GISS-E2-H","GISS-E2-R",\
    "HadGEM2-ES","IPSL-CM5A-LR","IPSL-CM5A-MR",\
    "IPSL-CM5B-LR","MIROC5","MIROC-ESM","MPI-ESM-LR",\
    "MPI-ESM-MR","MPI-ESM-P","MRI-CGCM3","NorESM1-M"]
model_length = {"ACCESS-10":150,"ACCESS-13":151,"BCC-CSM-11":150,\
    "BCC-CSM-11m":150,"CanESM2":150,\
    "CCSM4":151,"CNRM-CM5":150,"CNRM-CM5-2":140,\
    "CSIRO-Mk360":150,"GFDL-CM3":150,"GFDL-ESM2G":300,\
    "GFDL-ESM2M":300,"GISS-E2-H":151,"GISS-E2-R":151,\
    "HadGEM2-ES":151,"IPSL-CM5A-LR":260,\
    "IPSL-CM5A-MR":140,"IPSL-CM5B-LR":160,"MIROC5":151,\
    "MIROC-ESM":150,"MPI-ESM-LR":150,"MPI-ESM-MR":150,\
    "MPI-ESM-P":150,"MRI-CGCM3":150,"NorESM1-M":150}
    
# create array of final variables -- tas in {degC}; rsdt,rlut,rsut in {W/m2}
for var in ["tas","rsdt","rsut","rlut"]:
    exec(var+"_ctrl = np.zeros((300,len(model_list)),) * np.nan")
    exec(var+"_quad = np.zeros((300,len(model_list)),) * np.nan")

# load pre-processed CMIP5 results
for mod in model_list:
    for sim in ['ctrl','quad']:
        TMP = np.array([line for line in csv.reader(open('data/Climate_CMIP5/Climate_'+mod+'.'+sim+'_global.csv','r'))][1:], 
                       dtype=float)
        lgd = [line for line in csv.reader(open('data/Climate_CMIP5/Climate_'+mod+'.'+sim+'_global.csv','r'))][0]
        for var in ['tas','rsdt','rsut','rlut']:
            exec(var+'_'+sim+'[:len(TMP),model_list.index(mod)] = TMP[:,lgd.index(var)]')

In [ ]:
# THIS CODE BLOCK DEFINED A FUNCTION TO PLOT IRF
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_irf(ti, tf, func, ylim=None, ylabel=''):
    """
    Function that create a matplotlib figure and 
    plot a function 'func' between time 'ti' and 'tf' (in years)
    """  
    # create the figure
    nbcols  = int(np.log10(tf-ti))-1
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 3), layout="constrained")

    # add data on the figure           
    t = np.arange(ti, tf)
    ax.plot(t, func(t), ls='-', label='IRF')
    ax.plot(t, func(1e18*np.ones_like(t)), ls='--', label='a_C0')

    divider = make_axes_locatable(ax)

    # split the figure in several axis to zoom out progressively           
    i, t0 = 2, ti
    while ( (10**(i+1) <= tf) and (10**i >= ti) ):
        ax.axis([t0,10**i,0,1])

        ax = divider.append_axes("right", size=6/nbcols, pad=0.02, sharey=ax)
        ax.plot(t, func(t), ls='-', label='IRF')
        ax.plot(t, func(1e18*np.ones_like(t)), ls='--', label='limit')

        ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 1], ['','','','','',''])

        t0 = 10**i
        i += 1
    ax.axis([t0,tf,0,1])

    # polishing the figure
    fig.axes[0].set_xlabel("time (yr)", x=max(0.5, nbcols-1))
    fig.axes[0].set_ylabel(ylabel)

    if ylim is not None:
        fig.axes[0].set_ylim(ylim)
        fig.axes[0].set_yticks(np.linspace(ylim[0], ylim[1], 5))

    ax.legend(loc=0)
    
    return

#### Question 1:

* _Code the function IRF$_{tas}$ in the code block bellow and produce a figure to visualize it._

In [ ]:
# definition of the IRF for atmospheric CH4
def irf_tas_CH4(t):
    pass 
    # REPLACE THE "pass" STATEMENT BY YOUR IRF CODE

In [ ]:
# plotting the impulse response
# WRITE YOUR CODE HERE

* _Compare this latter to the previous results derived from CMIP5 models (Part II) ?_
   
<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>

#### Question 2:

* *Implement the function in Python in order to compute AGWP and AGTP for CH$_4$ and CO$_2$ for a time horizon ranging from 0 to 500 years.*

__Hint:__ We can assume that for methane change in radiative forcing responds linearly to rising CH$_4$ concentration (called $rad_\text{eff}$). 

In the last IPCC report, [Myrhe et al. (2013a)](https://acp.copernicus.org/articles/13/1853/2013/) use the following set of parameters :

In [ ]:
# CH4
tau_CH4 = 12.4 #{yr}
radeff_CH4 = 3.63E-4 #{W/m2 / ppb}
k_CH4 = k_CO2 * 16/44. #{TgCH4 / ppb}
# CO2
radeff_CO2 = 1.37E-5 #{W/m2 / ppb}

In [ ]:
# WRITE YOUR CODE HERE

#### Question 3:
* *Display $\text{GWP}(t)$ and $\text{GTP}(t)$ for methane.*

In [ ]:
# WRITE YOUR CODE HERE

#### Question 4:
* *Compute their values for $T_H$ = 20 years and 100 years and compare these latter to
those given in the last IPCC report (see Table below).*

| Metrics      |  20 years | 100 years |
|--------------|:---------:|:---------:|
| GWP$^{CH_4}$ |    84     |    28     |
| GTP$^{CH_4}$ |    67     |     4     |

In [ ]:
# WRITE YOUR CODE HERE

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>

#### Question 5:
* _Discuss the results and proposed a solution to account for missing processes._

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>

### 2.2 Assessing the GWP and GTP for other chemical species

In this section, we will investigate the role of other chemical species like the nitrous oxide (N$_2$O), the sulfur hexafluoride (SF$_6$), the black carbon (BC) and the sulfate (SO$_2$).

#### Question 6:
* _Characterize these chemical species (gas or aerosols, natural or anthropogenic,
long-lived or short-lived species)._

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>

In the following, we will rely on the set of parameters proposed in [Fuglestvedt et al. (2010)](https://doi.org/10.1016/j.atmosenv.2009.04.044)
and [Myrhe et al. (2013a)](https://acp.copernicus.org/articles/13/1853/2013/) :

In [ ]:
# N2O
tau_N2O = 121. #{yr}
radeff_N2O = 3.00E-3 #{W/m2 / ppb}
k_N2O = k_CO2 * 44/44. #{TgN2O / ppb}
# SF6
tau_SF6 = 3200. #{yr}
radeff_SF6 = 0.57 #{W/m2 / ppb}
k_SF6 = k_CO2 * 146/44. #{TgSF6 / ppb}
# SO2
tau_SO2 = 0.011 #{yr}
radeff_SO2 = -0.55 #{W/m2 / TgSO2}
# BC
tau_BC = 0.020 #{yr}
radeff_BC = 2.82 #{W/m2 / TgC}

#### Question 7:

* *Display the $\text{GWP}$ and $\text{GTP}$ for the various species.*

In [ ]:
# WRITE YOUR CODE HERE

* _Using a log-scale representation, compare $\text{GWP}$ and $\text{GTP}$._

In [ ]:
# WRITE YOUR CODE HERE

#### Question 8:

* _Discuss the potential limitation of this metrics and propose a conclusion for part III._

<font color="blue">
WRITE YOUR RESPONSE HERE !
</font>